You must do
```shell
pip install pywordseg
pip install overrides==4.1.2 regex
```

for Chinese segmentation to work.

In [1]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from pathlib import Path
from espnet2.text.phoneme_tokenizer import PhonemeTokenizer
from espnet2.text.token_id_converter import TokenIDConverter
from espnet2.bin.tts_inference import Text2Speech

from prosody.pinyin import *
from prosody.en_to_zh import PinyinArpaSpeech
%load_ext autoreload
%autoreload 2

PWD = %pwd
PWD = Path(PWD)
LJSPEECH_DIR = (PWD / '../egs2/ljspeech/tts1/').resolve()
device = 'cuda'

In [2]:
os.chdir(LJSPEECH_DIR)
pretrained_dir = LJSPEECH_DIR / "exp/tts_train_jets_raw_phn_tacotron_g2p_en_no_space"
pretrained_model_file = pretrained_dir / "train.total_count.ave_5best.pth"
pretrained_tts = Text2Speech.from_pretrained(
    train_config=pretrained_dir / "config_prosody.yaml",
    model_file=pretrained_model_file,
    device=device
)
pretrained_model = pretrained_tts.model
os.chdir(PWD)

config file: /home/perry/PycharmProjects/present/egs2/ljspeech/tts1/exp/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/config_prosody.yaml
Vocabulary size: 78
encoder self-attention layer type = self-attention
encoder self-attention layer type = self-attention


/home/perry/miniconda3/envs/espnet/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Initialize encoder.encoders.0.self_attn.linear_q.bias to zeros
Initialize encoder.encoders.0.self_attn.linear_k.bias to zeros
Initialize encoder.encoders.0.self_attn.linear_v.bias to zeros
Initialize encoder.encoders.0.self_attn.linear_out.bias to zeros
Initialize encoder.encoders.0.feed_forward.w_1.bias to zeros
Initialize encoder.encoders.0.feed_forward.w_2.bias to zeros
Initialize encoder.encoders.0.norm1.bias to zeros
Initialize encoder.encoders.0.norm2.bias to zeros
Initialize encoder.encoders.1.self_attn.linear_q.bias to zeros
Initialize encoder.encoders.1.self_attn.linear_k.bias to zeros
Initialize encoder.encoders.1.self_attn.linear_v.bias to zeros
Initialize encoder.encoders.1.self_attn.linear_out.bias to zeros
Initialize encoder.encoders.1.feed_forward.w_1.bias to zeros
Initialize encoder.encoders.1.feed_forward.w_2.bias to zeros
Initialize encoder.encoders.1.norm1.bias to zeros
Initialize encoder.encoders.1.norm2.bias to zeros
Initialize encoder.encoders.2.self_attn.linear_q

In [3]:
arpa_tokenizer = PhonemeTokenizer(g2p_type='g2p_en_no_space')
id_converter = TokenIDConverter(pretrained_tts.train_args.token_list)

In [66]:
save_dir = PWD / 'outputs/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/mandarin'
os.makedirs(save_dir, exist_ok=True)
pas = PinyinArpaSpeech(token_id_converter=id_converter, tts_inference_fn=pretrained_model.tts.generator.inference)

In [58]:
with_sandhi = hans_to_pinyins('眼艳烟')
reg_pinyins = regularize_pinyin(with_sandhi)

In [ ]:
import pandas as pd
from en_to_zh import INITALS, RIMES
transcript_file = data_dir / 'test/content_pinyin.txt'


def get_transcripts():
    transcripts = {}
    with open(transcript_file) as f:
        for line in f:
            wav_file, transcript = line.strip().split('\t', maxsplit=1)
            hans, pinyin_str = transcript.split('|', maxsplit=1)
            pinyin_str = ''.join(pinyin_str.split())
            transcripts[wav_file] = (hans, pinyin_str)
    return transcripts


transcripts = get_transcripts()
from prosody.pinyin import hans_to_pinyins

In [65]:
# Try on the first line of the AISHELL-3 dataset.
# line = 'SSB06930002.wav	武 wu3 术 shu4 始 shi3 终 zhong1 被 bei4 看 kan4 作 zuo4 我 wo3 国 guo2 的 de5 国 guo2 粹 cui4'
# custom_filename, chinese = line.strip().split(maxsplit=1)
# chinese = re.sub(r'[ a-z0-9]', '', chinese) + '。'

# chinese = '李万祥指了指墙上优秀教练的奖状'
# chinese = '江苏修鞋奶奶婉拒捐款一人养活患病老伴和儿子.'
# chinese = '且激吻戏竟然拍了三小时.'
# chinese = '我的大部分记忆是数据体'
# chinese = '仍然没有止住市场的下滑趋势'
# chinese = '四十四岁的徐女士报警称'
chinese = '等你生日就送给你'
custom_filename = chinese + '.wav'
save_dir = PWD / 'outputs/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/mandarin'
pac_update_dict = dict(pinyin_to_arpa_durations={
    'zhɨ': ('CH R UH1 R',
            '1 0.5 1 0'),
            'uang': ('W AH1 NG',
             '1 1 1'),
    'v': ('UH1 Y',
          '0.5 1.5'),
    've': ('Y UW1 EH1 ,',
           '0 0.5 1 0'),
    'r': ('R', '1'),
    'rɨ': ('R R',
            '1 1'),
    
})
# pac_update_dict = dict(pinyin_to_arpa_durations={'wo': ('W UH1 AO1', '0.8 0.8 0.8')})



inputs = pas.gen_audio(
    chinese,
    save_dir,
    verbose=False,
    overall_d_factor=8.0,
    fix_durations=True,
    arpa_in_filename=False,
    pac_update_dict=pac_update_dict,
    custom_filename=custom_filename,
    device=device,
)

Updated zhɨ, uang, v, ve, r, rɨ


In [20]:
pas.gen_inputs('SSB06930002.wav	武 wu3 术 shu4 始 shi3 终 zhong1 被 bei4 看 kan4 作 zuo4 我 wo3 国 guo2 的 de5 国 guo2 粹 cui4 .')

AssertionError: Mismatched hans and pinyins length: SSB.	武术始终被看作我国的国粹. ['06930002wav', 'wu3', 'shu4', 'shɨ3', 'zhong1', 'bei4', 'kan4', 'zuo4', 'wo3', 'guo2', 'de5', 'guo2', 'cui4']

In [72]:
from prosody.pinyin import VALID_PINYINS
# chinese = '思死死四四'
# custom_filename = chinese + '.wav'
save_dir = PWD / 'outputs/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/mandarin'
kwargs = dict(
    verbose=False,
    overall_d_factor=8.0,
    fix_durations=True,
    arpa_in_filename=False,
    # pac_update_dict=dict(pinyin_to_arpa_durations={'nve': ('UW1 EH1', '0.5 1'),}),
    pac_update_dict=dict(pinyin_to_arpa_durations={'ɨ': ('Z UH1', '0.5 2'),}),
    device=device
)

pinyins = []

for i, pinyin in enumerate(VALID_PINYINS):
    if pinyin.endswith('ɨ'):
        print(pinyin)
        pinyins.append(VALID_PINYINS[i] + str(i % 4 + 1))
        if i % 8 == 0:
            print(pinyins)
            inputs = pas.gen_audio(pinyins, save_dir, custom_filename=' '.join(pinyins) + '.wav', **kwargs)
            pinyins = []
print(pinyins)
inputs = pas.gen_audio(pinyins, save_dir, custom_filename=' '.join(pinyins) + '.wav', **kwargs)

zhɨ
['zhɨ1']
Updated ɨ
chɨ
shɨ
rɨ
zɨ
cɨ
sɨ
['chɨ2', 'shɨ3', 'rɨ4', 'zɨ1', 'cɨ2', 'sɨ3']
Updated ɨ


In [73]:
regularize_pinyin(hans_to_pinyins('次辞刺'))

['cɨ4', 'cɨ2', 'cɨ4']

In [68]:
from tqdm import tqdm
# Now run on entire dataset
# pinyin_file = (PWD / '../../datasets/data_aishell3/test/content_pinyin.txt').resolve()
pinyin_file = (PWD / '../../datasets/data_aishell3/test/content.txt').resolve()
# save_dir = PWD / 'outputs/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/aishell3'
save_dir = PWD / 'outputs/tts_train_jets_raw_phn_tacotron_g2p_en_no_space/aishell3_orig'
os.makedirs(save_dir, exist_ok=True)
with open(pinyin_file) as f:
    lines = f.read().splitlines()
    for line in tqdm(lines):
        filename, chinese = line.strip().split('\t', maxsplit=1)
        if (save_dir / filename).exists():
            continue
        # hans, pinyin_str = chinese.split('|', maxsplit=1)
        # pinyins = regularize_pinyin(pinyin_str.split())
        inputs = pas.gen_audio(
            # pinyins,
            chinese,
            save_dir,
            verbose=False,
            # disable_tones=True,
            inputs=None,
            # pac_update_dict=pas_update_dict,
            # infer_overrides=infer_overrides,
            overall_d_factor=8.0,
            fix_durations=True,
            arpa_in_filename=False,
            custom_filename=filename,
            # add_full_stop=True,
            device=device,
        )
        # arpas, tones, d_factor, d_split_factor, pitch_values, p_mod_fns = inputs


100%|██████████| 24773/24773 [1:43:52<00:00,  3.97it/s]  


In [69]:
import time
time.sleep(2000)
# Try again without tones
nopitch_dir = str(save_dir) + '_nopitch'
os.makedirs(nopitch_dir, exist_ok=True)
with open(pinyin_file) as f:
    lines = f.read().splitlines()
    for line in tqdm(lines):
        filename, chinese = line.strip().split('\t', maxsplit=1)
        # hans, pinyin_str = chinese.split('|', maxsplit=1)
        # pinyins = regularize_pinyin(pinyin_str.split())

        inputs = pas.gen_audio(
            chinese,
            nopitch_dir,
            verbose=False,
            disable_tones=True,
            inputs=None,
            # pac_update_dict=pas_update_dict,
            infer_overrides=None,
            overall_d_factor=8.0,
            fix_durations=True,
            arpa_in_filename=False,
            custom_filename=filename,
            device=device,
        )


100%|██████████| 24773/24773 [1:55:02<00:00,  3.59it/s]  


In [71]:
# Try again without durations
nodur_dir = nopitch_dir + '_nodur'
os.makedirs(nodur_dir, exist_ok=True)
with open(pinyin_file) as f:
    lines = f.read().splitlines()
    for line in tqdm(lines):
        filename, chinese = line.strip().split('\t', maxsplit=1)
        # hans, pinyin_str = chinese.split('|', maxsplit=1)
        # pinyins = regularize_pinyin(pinyin_str.split())

        inputs = pas.gen_audio(
            chinese,
            nodur_dir,
            verbose=False,
            disable_tones=True,
            disable_durations=True,
            inputs=None,
            # pac_update_dict=pas_update_dict,
            infer_overrides=None,
            overall_d_factor=8.0,
            fix_durations=True,
            arpa_in_filename=False,
            custom_filename=filename,
            device=device,
        )


100%|██████████| 24773/24773 [2:07:59<00:00,  3.23it/s]  


In [ ]:
use_utterances = '''
SSB11260212.wav	中国人现在是美国房地产遥遥领先的最大外国买家|zhong1 guo2 ren2 xian4 zai4 shi4 mei3 guo2 fang2 di4 chan3 yao2 yao2 ling3 xian1 de5 zui4 da4 wai4 guo2 mai3 jia4

SSB11260231.wav	比较节能的产品才会有补贴|bi3 jiao4 jie2 neng2 de5 chan2 pin3 cai2 hui4 you2 bu3 tie1

SSB11350152.wav	一年以来持续攀升的待售面积也出现首次下降|yi4 nian2 yi3 lai2 chi2 xu4 pan1 sheng1 de5 dai4 shou4 mian4 ji1 ye3 chu1 xian4 shou3 ci4 xia4 jiang4

SSB13220045.wav	这头大象你是不可能说得动的|zhe4 tou2 da4 xiang4 ni3 shi4 bu4 ke3 neng2 shuo1 de5 dong4 de5

SSB14520196.wav	一般跳绳双脚离地高度为二点五公分到五公分|yi4 ban1 tiao4 sheng2 shuang1 jiao3 li2 di4 gao1 du4 wei2 er4 dian2 wu3 gong1 fen1 dao4 wu3 gong1 fen1

SSB17390336.wav	未来这两幅相邻地块很可能同时开发|wei4 lai2 zhe4 liang3 fu2 xiang1 lin2 di4 kuai4 hen2 ke3 neng2 tong2 shi2 kai1 fa1

SSB17390210.wav	中秋节不回来就算了|zhong1 qiu1 jie2 bu4 hui2 lai2 jiu4 suan4 le5

SSB17590214.wav	告诉我他对我抱有极大的信心|gao4 su4 wo3 ta1 dui4 wo3 bao4 you3 ji2 da4 de5 xin4 xin1

SSB16300072.wav	意识到束缚才能享受无期徒刑中清醒的快乐|yi4 shi2 dao4 shu4 fu4 cai2 neng2 xiang3 shou4 wu2 qi1 tu2 xing2 zhong1 qing1 xing3 de5 kuai4 le4

SSB07020392.wav	第一届全国青年运动会射击比赛进入第四天|di4 yi1 jie4 quan2 guo2 qing1 nian2 yun4 dong4 hui4 she4 ji1 bi3 sai4 jin4 ru4 di4 si4 tian1

SSB07020429.wav	我们之间的差异再也不像我们的长相这样差距如此大|wo3 men5 zhi1 jian1 de5 cha1 yi2 zai4 ye3 bu2 xiang4 wo3 men5 de5 zhang3 xiang4 zhe4 yang4 cha1 ju4 ru2 ci3 da4

SSB13020205.wav	什么时候开始考虑出国留学|shen2 me5 shi2 hou4 kai1 shi2 kao3 lv4 chu1 guo2 liu2 xue2

SSB07020005.wav	你猜咱们这位慈善的买卖人现在开着什么|ni3 cai1 zan2 men5 zhe4 wei4 ci2 shan4 de5 mai3 mai4 ren2 xian4 zai4 kai1 zhe5 shen3 me5

SSB10550102.wav	没吃台湾麻辣锅就等于没来过台湾|mei2 chi1 tai2 wan1 ma2 la4 guo1 jiu4 deng3 yu2 mei2 lai2 guo4 tai2 wan1

SSB10550022.wav	该超市的一工作人员不断地安慰顾客|gai1 chao1 shi4 de5 yi4 gong1 zuo4 ren2 yuan2 bu2 duan4 de5 an1 wei4 gu4 ke4
'''

In [37]:
from pypinyin import pinyin as convert, STYLE_TONE3
from prosody.utils.utils import print_table
from pathlib import Path
PWD = %pwd
transcript_file = (Path(PWD) / '../../datasets/data_aishell3/test/content.txt').resolve()
import regex as re
with open(transcript_file) as f:
    lines = f.read().splitlines()
    # lines = ['SSB07360067.wav	认 ren4 识 shi5 东 dong1 西 xi5 丈 zhang4 夫 fu5 个 ge5 仍 reng1 然 ran2 血 xue3 友 you3 谊 yi2']
    skip_chars = list('什吗哦') 
    skip_prons = ['血 xue3', '谊 yi2', '个 ge5', '仍 reng1', '匙 shi2' '抚 fu2', '坯 pei1', '蕾 lei2', '息 xi5', '识 shi5', '辑 ji4', '符 fu3', '骰 shai3', '亚 ya3', '嫉 ji4', '坊 fang3', '违 wei3']
    skip_2_prons = ['东西 xi5', '丈夫 fu5', '姑娘 niang5', '明白 bai5', '先生 sheng5', '气氛 fen5', '面积 ji5', '好好 hao1', '时候 hou5' '政府 fu3', '项目 mu5', '部分 fen5', '莱坞 wu1', '分析 xi5', '然而 er3', '耳朵 duo1', '事情 qing5', '父亲 qin5', '宝宝 bao1']
    for line in lines:
        filename, chinese = line.split('\t', maxsplit=1)
        hans = list(re.sub(r'\P{Han}', '', chinese))
        orig_pinyins = re.sub(r'\p{Han}', '', chinese).strip().split()
        norm_pinyins = []
        for p in orig_pinyins:
            if p[-2] == 'r' and not p.startswith('er'):
                norm_pinyins.append(p[:-2] + p[-1])
                norm_pinyins.append('er2')
            else:
                norm_pinyins.append(p)
        possible_pinyins = convert(hans, heteronym=True, style=STYLE_TONE3, neutral_tone_with_five=True)
        corrections, errors = [], []
        for i, (han, p, possibles) in enumerate(zip(hans, norm_pinyins, possible_pinyins)):
            if p not in possibles:
                if p.endswith('2') and f'{p[:-1]}3' in possibles:
                    if i < len(hans)-1 and norm_pinyins[i+1].endswith('3'):
                        continue
                    elif i < len(hans) - 2 and norm_pinyins[i+1].endswith('2') and norm_pinyins[i+2].endswith('3'):
                        continue
                if han in skip_chars or f'{han} {p}' in skip_prons:
                    continue
                elif i and f'{hans[i-1]}{han} {p}' in skip_2_prons:
                    continue
                if p.endswith('5'):
                    if han == hans[-1]:
                        continue
                    elif i and hans[i-1] == han:
                        continue
                if p[0] in ['z', 'c', 's'] and p[0] + 'h' + p[1:] in possibles:
                    old = f'{han} {p}'
                    new = f'{han} {p[0]}h{p[1:]}'
                    corrections.append((old, new))
                elif p[0:2] in ['zh', 'ch', 'sh'] and p[0] + p[2:] in possibles:
                    old = f'{han} {p}'
                    new = f'{han} {p[0]}{p[2:]}'
                    corrections.append((old, new))
                elif p[-3:-1] == 'ng' and p[:-2] + p[-1] in possibles:
                    corrections.append((f'{han} {p}', f'{han} {p[:-2]}{p[-1]}'))
                elif p[-2:-1] == 'n' and p[:-1] + 'g' + p[-1] in possibles:
                    corrections.append((f'{han} {p}', f'{han} {p[:-1]}g{p[-1]}'))
                elif p[:-1] == 'e' and p[0] + 'r' + p[-1] in possibles:
                    corrections.append((f'{han} {p}', f'{han} {p[0]}r{p[-1]}'))
                elif p[-2:-1] == 'i' and p[:-2] + 'u' + p[-1] in possibles:
                    corrections.append((f'{han} {p}', f'{han} {p[:-2]}u{p[-1]}'))
                elif p[-3:-1] == 'in' and p[:-3] + 'un' + p[-1] in possibles:
                    corrections.append((f'{han} {p}', f'{han} {p[:-3]}un{p[-1]}'))
                elif p[-2:-1] == 'i' and p[:-2] + 'v' + p[-1] in possibles:
                    corrections.append((f'{han} {p}', f'{han} {p[:-2]}v{p[-1]}'))
                elif p[-3:-1] == 'ie' and p[:-3] + 'ue' + p[-1] in possibles:
                    corrections.append((f'{han} {p}', f'{han} {p[:-3]}ue{p[-1]}'))
                elif p[-3:-1] == 'ie' and p[:-3] + 've' + p[-1] in possibles:
                    corrections.append((f'{han} {p}', f'{han} {p[:-3]}ve{p[-1]}'))
                elif p[:-1] == 'yan' and 'yuan' + p[-1] in possibles:
                    corrections.append((f'{han} {p}', f'{han} yuan{p[-1]}'))
                elif p[1:-1] == 'ian' and p[0] + 'uan' + p[-1] in possibles:
                    corrections.append((f'{han} {p}', f'{han} {p[0]}uan{p[-1]}'))
                elif p[0] == 'l' and 'n' + p[1:] in possibles:
                    corrections.append((f'{han} {p}', f'{han} n{p[1:]}'))
                elif p[0] == 'n' and 'l' + p[1:] in possibles:
                    corrections.append((f'{han} {p}', f'{han} l{p[1:]}'))
                elif p[0] == 'l' and 'r' + p[1:] in possibles:
                    corrections.append((f'{han} {p}', f'{han} r{p[1:]}'))
                errors.extend([han, p, str(possibles)])
        text = line
        if corrections:
            # print(corrections)
            for old, new in corrections:
                text = text.replace(old, new)
        if errors:
            print(' '.join(errors), text)
            # SSB14370056

灵 ling5 ['ling2'] SSB07110361.wav	卡 ka3 瑞 rui4 娜 na4 看 kan4 起 qi3 来 lai2 挺 ting3 机 ji1 灵 ling5 的 de5
楚 chu5 ['chu3'] SSB07170051.wav	先 xian1 想 xiang3 清 qing1 楚 chu5 再 zai4 行 xing2 动 dong4
绅 shen2 ['shen1'] SSB07170367.wav	全 quan2 聚 ju4 德 de2 烤 kao3 鸭 ya1 店 dian4 绅 shen2 士 shi4 海 hai3 店 dian4
会 hui3 ['hui4', 'kuai4'] SSB07360190.wav	等 deng2 会儿 huir3 记 ji4 住 zhu4 把 ba3 账 zhang4 单 dan1 给 gei2 我 wo3
补 bu2 ['bu3'] SSB07360253.wav	美 mei3 国 guo2 巴 ba1 拿 la2 马 ma3 南 nan2 方 fang1 司 si1 令 ling4 部 bu4 给 gei2 我 wo3 们 men5 送 song4 了 le5 补 bu2 金 jin1
匙 shi2 ['shi5', 'chi2'] SSB07360318.wav	对 dui4 我 wo3 车 che1 钥 yao4 匙 shi2
指 zhi5 ['zhi3', 'zhi1', 'zhi2'] SSB07020032.wav	这 zhe4 个 ge4 戒 jie4 指 zhi5 是 shi4 我 wo2 母 mu3 亲 qin1 临 lin2 终 zhong1 前 qian2 留 liu2 给 gei2 我 wo3 的 de5
抚 fu2 ['fu3'] SSB07020178.wav	我 wo3 们 men5 尽 jin3 一 yi2 切 qie4 能 neng2 力 li4 抚 fu2 养 yang2 你 ni2 长 zhang3 大 da4
即 ji4 ['ji2'] SSB07020424.wav	即 ji4 使 shi3 还 hai2 还 huan2 那 na4 笔 bi3 钱 qian2 需 xu1 要 yao4 一 yi4 千 qian1 年 nian2 时 shi2 

In [43]:
from pypinyin import lazy_pinyin, STYLE_TONE3, pinyin as convert
# pypinyin is not good at 多音字, so we merge original pinyins with pypinyin to get the reference pinyins  
pinyin_file = (Path(PWD) / '../../datasets/data_aishell3/test/content_pinyin.txt').resolve()
with open(transcript_file) as f:
    with open(pinyin_file, 'w') as out_f:
        for line in f:
            filename, chinese = line.split('\t', maxsplit=1)
            hans = re.sub(r'\P{Han}', '', chinese)
            orig_pinyins = re.sub(r'\p{Han}', '', chinese).strip().split()
            no_erhua = []
            for pinyin in orig_pinyins:  # expand erhua
                m = GENERIC_ERHUA_REGEX.fullmatch(pinyin)
                if m and not pinyin.startswith('er'):
                    no_erhua.append(''.join(m.groups()))
                    no_erhua.append('er2')
                else:
                    no_erhua.append(pinyin)
            real_pinyins = lazy_pinyin(hans, style=STYLE_TONE3, neutral_tone_with_five=True)
            true_pinyins = []
            for i, (han, orig, real) in enumerate(zip(hans, orig_pinyins, real_pinyins)):
                use_orig = True
                if orig != real:
                    possibles = convert(han, heteronym=True, style=STYLE_TONE3, neutral_tone_with_five=True)[0]
                    if orig not in possibles:
                        if orig[-1] == '2' and f'{orig[:-1]}3' in possibles:
                            if i < len(hans)-1 and orig_pinyins[i+1].endswith('3'):
                                pass
                            elif i < len(hans) - 2 and orig_pinyins[i+1].endswith('2') and orig_pinyins[i+2].endswith('3'):
                                pass
                            else:
                                use_orig = False
                        else:
                            use_orig = False
                true_pinyins.append(orig if use_orig else real)
            out_f.write(f'{filename}\t{hans}|{" ".join(true_pinyins)}\n')

In [ ]:
import regex as re 
filename = '/home/perry/PycharmProjects/datasets/data_aishell3/test/content.txt'
old_name = filename + '.bak'
import shutil
shutil.copy(filename, old_name)
corr_file = '/home/perry/PycharmProjects/present/prosody/tmp.txt' 
corrs = {}
with open(corr_file) as f:
    for line in f:
        repl = line.split('SSB')[1]
        wav, repl = repl.split('\t')
        wav = 'SSB' + wav
        corrs[wav] = repl
        
with open(filename, 'w') as f:
    with open(old_name) as old_f:
        for line in old_f:
            wav, text = line.split('\t')
            if wav in corrs:
                f.write(f'{wav}\t{corrs[wav]}')
            else:
                f.write(line)